# CoreUtil nb

## This nb bundles together core utilitiy functionality such as job configuration

#### Logger

In [1]:
import logging
import sys
logger = logging.getLogger()
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.DEBUG)

#### Import core libraries

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import tables 
import numpy as np

#### inline plots

In [3]:
%matplotlib inline

In [4]:
%reload_ext autoreload
%autoreload 2

### Import notebooks

In [5]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [6]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [7]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [8]:
sys.meta_path.append(NotebookFinder())

In [9]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [11]:
import shutil
from IPython.paths import get_ipython_package_dir


In [14]:
get_ipython_package_dir()

u'/usr/local/lib/python2.7/site-packages/IPython'

In [17]:
os.path

<module 'posixpath' from '/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/posixpath.pyc'>

In [ ]:

utils = os.path.join(get_ipython_package_dir(), 'utils')
shutil.copy(os.path.join("nbpackage", "mynotebook.ipynb"),
            os.path.join(utils, "inside_ipython.ipynb")
)

In [ ]:
nbDiomira.configure_path('DIOMIRA_NA_ZLIB_test2.csv', '../../Config/')

In [ ]:

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())


### job configuration

The main function is *configure* which takes the full path to a CSV file and two variables, INFO and level.

INFO prints an "informative" text (e.g, a fragment of invisible cities) when TRUE

level sets the level of the logger

An example of a configuration file follows

In [ ]:
less ../../Config/DIOMIRA_NA_ZLIB_test2.csv

In [ ]:

def cdf_to_dict(cdf):
    """
    transforms the configuration data frame into a dictionary
    """

    dc ={}
    for k in cdf.keys():
        dc[k] = cdf[k][0]
    return dc
    

def configure(cfile, INFO=False, level='INFO'):

    """
    Configures job
    """
    
    lg = 'logging.'+level
    logger.setLevel(eval(lg))

    if cfile == '':
        print("Path to configuration file not given. Please specify path")
        
    cfp =pd.read_csv(cfile,comment="#")
    CFP = cdf_to_dict(cfp)
    
    logger.info("Configuration Parameters (CFP) dictionary  = {}".format(CFP))
    return INFO, CFP
    


In [ ]:
configure('../../Config/DIOMIRA_NA_ZLIB_test2.csv',INFO=True,level='INFO')

### Utility functions

#### array manipulation

In [ ]:
def farray_from_string(sfl):
    """
    returns a np array of floats from a string (sfl)
    representing the array in which the numbers are separated by blanks
    e.g, '1.4 3.6 6.7' -->np.array(1.4,3.6,6.7)
    """
    sarr = sfl.split(' ')
    arr = []
    for x in sarr:
        arr.append(float(x))
    return np.array(arr)

In [ ]:
farray_from_string('1 2 3 4')

In [ ]:
def rebin_array(a, stride):
    """
    rebins the array according to stride 
    """
    lenb = len(a)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

In [ ]:
x = np.arange(0, 3*np.pi, 0.01)
y = np.sin(x)

In [ ]:
plt.plot(x,y)

In [ ]:
len(x)

In [ ]:
np.sum(y)

In [ ]:
xrb = rebin_array(x, 10)
yrb = rebin_array(y, 10)

In [ ]:
plt.plot(xrb,yrb)

In [ ]:
len(xrb)

In [ ]:
np.sum(yrb)